In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import scipy.stats

In [2]:
# 0
intakes = pd.read_csv("preprocessed_AnimalCenter-AustinTX_Intakes.csv")
outcomes = pd.read_csv("preprocessed_AnimalCenter-AustinTX_Outcomes.csv")
covid19 = pd.read_csv("preprocessed_Covid19-Texas.csv")

In [3]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132577 entries, 0 to 132576
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   animal_id      132577 non-null  object 
 1   name           132577 non-null  object 
 2   date_of_birth  132577 non-null  object 
 3   out_type       132577 non-null  object 
 4   out_subtype    132577 non-null  object 
 5   animal_type    132577 non-null  object 
 6   out_sex        132577 non-null  object 
 7   out_age        132577 non-null  object 
 8   breed          132577 non-null  object 
 9   color          132577 non-null  object 
 10  out_date       132577 non-null  object 
 11  out_time1      132577 non-null  object 
 12  out_time2      132577 non-null  object 
 13  out_age2       132577 non-null  float64
dtypes: float64(1), object(13)
memory usage: 14.2+ MB


In [4]:
# 1
intakes["in_date"] = pd.to_datetime(intakes["in_date"], format="%Y-%m-%d")
outcomes["out_date"] = pd.to_datetime(outcomes["out_date"], format="%Y-%m-%d")
covid19["date"] = pd.to_datetime(covid19["date"], format="%Y-%m-%d")


In [5]:
# 2
intakes = intakes.loc[intakes["in_date"] > datetime(2020, 3, 25), ["in_date", "animal_id"]].copy()
outcomes = outcomes.loc[outcomes["out_date"] > datetime(2020, 3, 25), ["out_date", "animal_id"]].copy()
covid19 = covid19.loc[covid19["confirmed"]>0, ["date", "confirmed", "deaths"]].copy()

In [6]:
# 3
intakes.rename(columns={"in_date":"date", "animal_id":"intakes"}, inplace=True)
outcomes.rename(columns={"out_date":"date", "animal_id":"outcomes"}, inplace=True)

In [7]:
# 4
intakes = intakes.groupby(by="date").count()
outcomes = outcomes.groupby(by="date").count()
data1 = intakes.merge(outcomes, how="left", on="date")

In [8]:
# 5
data = data1.merge(covid19, how="left", on="date")

In [9]:
# 6
data["intakes"].fillna(0, inplace=True)
data["outcomes"].fillna(0, inplace=True)
data["confirmed"].fillna(0, inplace=True)
data["deaths"].fillna(0, inplace=True)

data["intakes"] = data["intakes"].astype("int")
data["outcomes"] = data["outcomes"].astype("int")
data["confirmed"] = data["confirmed"].astype("int")
data["deaths"] = data["deaths"].astype("int")

In [10]:
# 7
data.to_csv("created_SampleForAnalysis.csv", index=False)

In [11]:
data

,date,intakes,outcomes,confirmed,deaths
0,2020-03-26,17,16,1,0
1,2020-03-27,5,16,1,0
2,2020-03-28,13,22,1,0
3,2020-03-29,8,14,2,0
4,2020-03-30,15,9,2,0
...,...,...,...,...,...
564,2021-10-13,50,39,3413,51
565,2021-10-14,40,25,3427,51
566,2021-10-15,78,37,3432,51
567,2021-10-16,25,23,3437,51


In [12]:
#data2 = data[['intakes','outcomes','confirmed','deaths']]
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data2.to_numpy())
# data_scaled = pd.DataFrame(data_scaled, columns=['intakes','outcomes','confirmed','deaths'])

In [13]:
scipy.stats.pearsonr(data["intakes"], data["confirmed"])

(0.4148119974805127, 4.516981982393535e-25)

In [14]:
scipy.stats.pearsonr(data["confirmed"], data["outcomes"])

(0.45422367340487113, 2.5966464838737226e-30)